In [66]:
import pandas as pd
import sqlite3

In [67]:
icao = pd.read_csv('ICAO_airports.csv')
top = pd.read_csv('top_airports.csv')

1) Use the data supplied of airport codes
(ICAO_airports.csv) and top airports (top_airports.csv)
to make a table of 50 most travelled airports in the US
and containing relevant information, like name, wikipedia
link, latitude, longitude.

In [68]:
!rm airports.db

connection = sqlite3.connect("airports.db")

cursor = connection.cursor()

sql_cmd = """CREATE TABLE icao 
            (iid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            name TEXT, 
            wikipedia_link TEXT, 
            iata TEXT, 
            latitude_deg FLOAT, 
            longitude_deg FLOAT)"""

cursor.execute(sql_cmd)

sql_cmd = """CREATE TABLE top 
            (tid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            name TEXT, 
            iata TEXT)"""

cursor.execute(sql_cmd)

In [69]:
#icao
#top

In [70]:
connection.commit()
#connection.close()

In [71]:
icao = icao.where(pd.notnull(icao), None)
for ii, row in icao.iterrows():
    iparams = (row['name'], row['wikipedia_link'], row['iata_code'], row['latitude_deg'], row['longitude_deg'])
    sql_cmd = """INSERT INTO icao
                (name, wikipedia_link, iata, latitude_deg, longitude_deg)
                VALUES (?, ?, ?, ?, ?)"""
    
    cursor.execute(sql_cmd, iparams)

In [72]:
connection.commit()
#connection.close()

In [73]:
print(top.columns)

Index(['City', 'FAA', 'IATA', 'ICAO', 'Airport', 'Role', 'Enplanements'], dtype='object')


In [74]:
top = top.where(pd.notnull(top), None)
for ii, row in top.iterrows():
    iparams = (row['Airport'], row['IATA'])
    sql_cmd = """INSERT INTO top
                (name, iata)
                VALUES (?, ?)"""
    
    cursor.execute(sql_cmd, iparams)

In [75]:
connection.commit()

In [76]:
# sql_cmd = "SELECT * FROM icao WHERE longitude_deg > 0"
# cursor.execute(sql_cmd)

# db_info = cursor.fetchall()
# for entry in db_info:
#     print(entry)

In [77]:
# Fix... figure out how to save this.
sql_cmd = """SELECT top.name, top.iata, icao.wikipedia_link, icao.latitude_deg, icao.longitude_deg
            FROM top LEFT JOIN icao ON
            top.iata = icao.iata"""

cursor.execute(sql_cmd)
db_info = cursor.fetchall()
print(len(db_info))
for entry in db_info: 
    print(entry)

50
('Hartsfield-Jackson Atlanta International Airport', 'ATL', 'http://en.wikipedia.org/wiki/Hartsfield-Jackson_Atlanta_International_Airport', 33.6366996765137, -84.4281005859375)
("Chicago O'Hare International Airport", 'ORD', "http://en.wikipedia.org/wiki/O'Hare_International_Airport", 41.97859955, -87.90480042)
('Los Angeles International Airport', 'LAX', 'http://en.wikipedia.org/wiki/Los_Angeles_International_Airport', 33.94250107, -118.4079971)
('Dallas/Fort Worth International Airport', 'DFW', 'http://en.wikipedia.org/wiki/Dallas-Fort_Worth_International_Airport', 32.896800994873, -97.0380020141602)
('Denver International Airport', 'DEN', 'http://en.wikipedia.org/wiki/Denver_International_Airport', 39.8616981506348, -104.672996520996)
('John F. Kennedy International Airport', 'JFK', 'http://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport', 40.63980103, -73.77890015)
('San Francisco International Airport', 'SFO', 'http://en.wikipedia.org/wiki/San_Francisco_Internation

2) Build another table that will hold historical weather information: min/max temperature, relative humidity, and precipitation.

In [ ]:
sql_cmd = """CREATE TABLE weather_tmin 
            (wid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            da TEXT, 
            iata TEXT)"""

# cursor.execute(sql_cmd)

sql_cmd = """CREATE TABLE weather_tmin 
            (wid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            da TEXT, 
            iata TEXT)"""

# cursor.execute(sql_cmd)

sql_cmd = """CREATE TABLE weather_tmin 
            (wid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            da TEXT, 
            iata TEXT)"""

# cursor.execute(sql_cmd)

sql_cmd = """CREATE TABLE weather_tmin 
            (wid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            da TEXT, 
            iata TEXT)"""

# cursor.execute(sql_cmd)

sql_cmd = """CREATE TABLE weather_tmin 
            (wid INTEGER  NOT NULL  PRIMARY KEY  AUTOINCREMENT DEFAULT 0, 
            da TEXT, 
            iata TEXT)"""

# cursor.execute(sql_cmd)

3) Using the data from the xarray/netCDF4 example in class, populate the database from 1990-2000.

In [113]:
import xarray as xr
import numpy as np

In [115]:
# temp: tasmax/tasmin, RH: rhsmax/rhsmin, precipitation: pr
# downward shortwave solar radiation(rsds), eastward wind(uas), 
# northward wind(vas), and specific humidity(huss)

datatype = "tasmax" 

data_path = ("http://thredds.northwestknowledge.net:8080/"
             f"thredds/dodsC/agg_macav2metdata_{datatype}"
             "_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc"
            )

max_temp_xr_all = xr.open_dataset(data_path)  
max_temp_xr = max_temp_xr_all['air_temperature'].sel(time=slice("1990", "2000"))
max_temp_xr

<xarray.DataArray 'air_temperature' (time: 132, lat: 585, lon: 1386)>
[107026920 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 235.2 235.3 235.3 235.4 ... 292.8 292.9 292.9 292.9
  * time     (time) object 1990-01-15 00:00:00 ... 2000-12-15 00:00:00
Attributes:
    long_name:      Monthly Average of Daily Maximum Near-Surface Air Tempera...
    units:          K
    grid_mapping:   crs
    standard_name:  air_temperature
    height:         2 m
    cell_methods:   time: maximum(interval: 24 hours);mean over days
    _ChunkSizes:    [ 10  44 107]

In [116]:
datatype = "tasmin" 

data_path = ("http://thredds.northwestknowledge.net:8080/"
             f"thredds/dodsC/agg_macav2metdata_{datatype}"
             "_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc"
            )

min_temp_xr_all = xr.open_dataset(data_path)  
min_temp_xr = min_temp_xr_all['air_temperature'].sel(time=slice("1990", "2000"))
min_temp_xr

<xarray.DataArray 'air_temperature' (time: 132, lat: 585, lon: 1386)>
[107026920 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 235.2 235.3 235.3 235.4 ... 292.8 292.9 292.9 292.9
  * time     (time) object 1990-01-15 00:00:00 ... 2000-12-15 00:00:00
Attributes:
    long_name:      Monthly Average of Daily Minimum Near-Surface Air Tempera...
    units:          K
    grid_mapping:   crs
    standard_name:  air_temperature
    height:         2 m
    cell_methods:   time: minimum(interval: 24 hours); mean over days
    _ChunkSizes:    [ 10  44 107]

In [126]:
datatype = "rhsmax" 

data_path = ("http://thredds.northwestknowledge.net:8080/"
             f"thredds/dodsC/agg_macav2metdata_{datatype}"
             "_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc"
            )

max_rhum_xr_all = xr.open_dataset(data_path) 
max_rhum_xr = max_rhum_xr_all['relative_humidity'].sel(time=slice("1990", "2000"))
max_rhum_xr

<xarray.DataArray 'relative_humidity' (time: 132, lat: 585, lon: 1386)>
[107026920 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 235.2 235.3 235.3 235.4 ... 292.8 292.9 292.9 292.9
  * time     (time) object 1990-01-15 00:00:00 ... 2000-12-15 00:00:00
Attributes:
    long_name:      Monthly Average of Surface Daily Maximum Relative Humidity
    units:          %
    grid_mapping:   crs
    standard_name:  relative_humidity
    height:         2 m
    cell_methods:   time: maximum(interval: 24 hours); mean over days
    comments:       This is the relative humidity with respect to liquid wate...
    _ChunkSizes:    [ 10  44 107]

In [124]:
datatype = "rhsmin" 

data_path = ("http://thredds.northwestknowledge.net:8080/"
             f"thredds/dodsC/agg_macav2metdata_{datatype}"
             "_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc"
            )

min_rhum_xr_all = xr.open_dataset(data_path) 
min_rhum_xr = min_rhum_xr_all['relative_humidity'].sel(time=slice("1990", "2000"))
min_rhum_xr

<xarray.DataArray 'relative_humidity' (time: 132, lat: 585, lon: 1386)>
[107026920 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 235.2 235.3 235.3 235.4 ... 292.8 292.9 292.9 292.9
  * time     (time) object 1990-01-15 00:00:00 ... 2000-12-15 00:00:00
Attributes:
    long_name:      Monthly Average of Surface Daily Minimum Relative Humidity
    units:          %
    grid_mapping:   crs
    standard_name:  relative_humidity
    height:         2 m
    cell_methods:   time: minimum(interval: 24 hours); mean over days
    comments:       This is the relative humidity with respect to liquid wate...
    _ChunkSizes:    [ 10  44 107]

In [128]:
datatype = "pr" 

data_path = ("http://thredds.northwestknowledge.net:8080/"
             f"thredds/dodsC/agg_macav2metdata_{datatype}"
             "_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc"
            )

precip_xr_all = xr.open_dataset(data_path) 
precip_xr = precip_xr_all['precipitation'].sel(time=slice("1990", "2000"))
precip_xr

<xarray.DataArray 'precipitation' (time: 132, lat: 585, lon: 1386)>
[107026920 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 235.2 235.3 235.3 235.4 ... 292.8 292.9 292.9 292.9
  * time     (time) object 1990-01-15 00:00:00 ... 2000-12-15 00:00:00
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    grid_mapping:   crs
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [138]:
precip_xr['time'].dt.year

<xarray.DataArray 'year' (time: 132)>
array([1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990,
       1990, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991,
       1991, 1991, 1992, 1992, 1992, 1992, 1992, 1992, 1992, 1992, 1992,
       1992, 1992, 1992, 1993, 1993, 1993, 1993, 1993, 1993, 1993, 1993,
       1993, 1993, 1993, 1993, 1994, 1994, 1994, 1994, 1994, 1994, 1994,
       1994, 1994, 1994, 1994, 1994, 1995, 1995, 1995, 1995, 1995, 1995,
       1995, 1995, 1995, 1995, 1995, 1995, 1996, 1996, 1996, 1996, 1996,
       1996, 1996, 1996, 1996, 1996, 1996, 1996, 1997, 1997, 1997, 1997,
       1997, 1997, 1997, 1997, 1997, 1997, 1997, 1997, 1998, 1998, 1998,
       1998, 1998, 1998, 1998, 1998, 1998, 1998, 1998, 1998, 1999, 1999,
       1999, 1999, 1999, 1999, 1999, 1999, 1999, 1999, 1999, 1999, 2000,
       2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000])
Coordinates:
  * time     (time) object 1990-01-15 00:00:00 ... 2000-12-15 00:00:00

4) For each pair of cities/airports determine how the daily change of temperature high and precipitation from one city predicts the daily change of the other city 1, 3, & 7 days in advance.

5) Plot the correlation strengths for the 10 top pairs for all three dates, for temperature and precipitation (separately) as a function of distance. Also make a plot as a function of longitude different. What trends do you see? (Note: for correlations between pairs, try np.corrcoef.)